In [1]:
import yfinance as yf
from datetime import datetime
from zoneinfo import ZoneInfo
import os

def compute_portfolio_change(stocks_dict, portfolio_name):
    total_weighted_change = 0
    total_weight = 0
    errors = []

    for name, (ticker, weight) in stocks_dict.items():
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            current = info.get("currentPrice")
            prev_close = info.get("previousClose")

            if current and prev_close:
                pct_change = ((current - prev_close) / prev_close) * 100
                total_weighted_change += pct_change * (weight / 100)
                total_weight += weight
            else:
                errors.append(f"{name} ({ticker}): missing price data")
        except Exception as e:
            errors.append(f"{name} ({ticker}): {e}")

    if total_weight > 0:
        summary = f"📊 {portfolio_name} Portfolio % Change: {total_weighted_change:.2f}%"
    else:
        summary = f"⚠️ {portfolio_name}: Could not compute weighted average."

    return summary, errors


# --- Motilal Oswal Portfolio ---
motilal_oswal = {
    "Coforge": ("COFORGE.NS", 10.13),
    "Persistent Systems": ("PERSISTENT.NS", 9.74),
    "Kalyan Jewellers": ("KALYANKJIL.NS", 8.02),
    "Polycab India": ("POLYCAB.NS", 4.93),
    "Bharti Hexacom": ("BHARTIHEXA.NS", 4.06),
    "Dixon Technologies": ("DIXON.NS", 4.04),
    "Trent": ("TRENT.NS", 3.98),
    "One 97 Communications": ("PAYTM.NS", 3.78),
    "Max Healthcare": ("MAXHEALTH.NS", 3.69),
    "KEI Industries": ("KEI.NS", 3.27),
    "ABB India": ("ABB.NS", 2.22),
    "KPIT Technologies": ("KPITTECH.NS", 2.20),
    "Tube Investments": ("TIINDIA.NS", 2.12),
    "Indian Hotels": ("INDHOTEL.NS", 2.01),
    "Jindal Steel": ("JINDALSTEL.NS", 2.00),
    "Prestige Estates": ("PRESTIGE.NS", 1.93),
    "Mazagon Dock": ("MAZDOCK.NS", 1.83),
    "Balkrishna Industries": ("BALKRISIND.NS", 1.32),
    "Phoenix Mills": ("PHOENIXLTD.NS", 0.88),
    "Supreme Industries": ("SUPREMEIND.NS", 0.88),
    "Mankind Pharma": ("MANKIND.NS", 0.75),
}

# --- Quant Small Cap Portfolio ---
quant_small_cap = {
    "Reliance Industries": ("RELIANCE.NS", 9.85),
    "Jio Financial Services": ("JIOFIN.NS", 5.69),
    "Aegis Logistics": ("AEGISLOG.NS", 3.72),
    "Bikaji Foods": ("BIKAJI.NS", 3.05),
    "RBL Bank": ("RBLBANK.NS", 2.86),
    "Poly Medicure": ("POLYMED.NS", 2.56),
    "HFCL": ("HFCL.NS", 2.35),
    "Sun TV Network": ("SUNTV.NS", 2.30),
    "Adani Enterprises": ("ADANIENT.NS", 2.21),
    "Adani Power": ("ADANIPOWER.NS", 2.18),
    "Aster DM Healthcare": ("ASTERDM.NS", 2.11),
    "Bayer Cropscience": ("BAYERCROP.NS", 1.99),
    "Arvind Ltd": ("ARVIND.NS", 1.79),
    "Pfizer Ltd": ("PFIZER.NS", 1.74),
    "NBCC (India)": ("NBCC.NS", 1.63),
    "Piramal Enterprises": ("PEL.NS", 1.59),
    "Sequent Scientific": ("SEQUENT.NS", 1.53),
    "Anand Rathi Wealth": ("ANANDRATHI.NS", 1.48),
    "Welspun Corp": ("WELCORP.NS", 1.43),
    "Minda Corporation": ("MINDACORP.NS", 1.35),
    "NCC Ltd": ("NCC.NS", 1.28),
    "Jubilant Life Sciences": ("JUBLINGREA.NS", 1.23),
    "Welspun Enterprises": ("WELENT.NS", 1.20),
    "Keystone Realtors": ("RUSTOMJEE.NS", 1.19),
    "Just Dial": ("JUSTDIAL.NS", 1.19),
}

# --- PPFAS Portfolio ---
ppfas = {
    "HDFC Bank": ("HDFCBANK.NS", 8.11),
    "Bajaj Holdings": ("BAJAJHLDNG.BO", 6.87),
    "Coal India": ("COALINDIA.NS", 5.95),
    "Power Grid": ("POWERGRID.NS", 5.83),
    "ICICI Bank": ("ICICIBANK.NS", 4.87),
    "ITC": ("ITC.NS", 4.40),
    "Kotak Mahindra Bank": ("KOTAKBANK.NS", 3.95),
    "Maruti Suzuki": ("MARUTI.NS", 3.58),
    "Mahindra & Mahindra": ("M&M.NS", 3.51),
    "Facebook": ("META", 3.15),
    "Axis Bank": ("AXISBANK.NS", 3.13),
    "HCL Technologies": ("HCLTECH.NS", 2.89),
    "Microsoft": ("MSFT", 2.78),
    "Alphabet": ("GOOGL", 2.67),
    "Amazon": ("AMZN", 2.32),
    "Bharti Airtel": ("BHARTIARTL.NS", 2.11),
    "Infosys": ("INFY.NS", 1.40),
    "Zydus Lifesciences": ("ZYDUSLIFE.NS", 1.28),
    "Cipla": ("CIPLA.NS", 1.25),
    "Dr. Reddy's": ("DRREDDY.NS", 1.25),
    "Balkrishna Industries": ("BALKRISIND.NS", 1.00),
    "Reliance Industries": ("RELIANCE.NS", 0.89),
    "Indian Energy Exchange": ("IEX.NS", 0.85),
}

# -------- Build full report text --------
lines = []
lines.append("--- Portfolio Performance Summary ---")
now = datetime.now(ZoneInfo("Asia/Kolkata")).strftime("%Y-%m-%d %H:%M:%S")
lines.append(f"Report Generated On: {now}")
lines.append("")  # blank line

for name, portfolio in [
    ("Motilal Oswal", motilal_oswal),
    ("Quant Small Cap", quant_small_cap),
    ("PPFAS", ppfas),
]:
    summary, errors = compute_portfolio_change(portfolio, name)
    lines.append(summary)
    if errors:
        lines.append("  Issues:")
        for e in errors:
            lines.append(f"    - {e}")
    lines.append("")  # blank line between portfolios

full_report = "\n".join(lines)

# -------- Save to output/result.txt for GitHub Pages / Shortcut --------
os.makedirs("output", exist_ok=True)

with open("output/result.txt", "w", encoding="utf-8") as f:
    f.write(full_report)

# Still print to notebook logs (useful in Actions logs)
print(full_report)


ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: PEL.NS"}}}


--- Portfolio Performance Summary ---
Report Generated On: 2025-12-09 14:37:53

📊 Motilal Oswal Portfolio % Change: -0.54%

📊 Quant Small Cap Portfolio % Change: 0.16%
  Issues:
    - Piramal Enterprises (PEL.NS): missing price data

📊 PPFAS Portfolio % Change: -0.27%
  Issues:
    - Facebook (META): missing price data
    - Microsoft (MSFT): missing price data
    - Alphabet (GOOGL): missing price data
    - Amazon (AMZN): missing price data

